# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 20 2022 12:47PM,237756,10,Eme-99809,Emerginnova,Released
1,May 20 2022 12:45PM,237755,20,8044783,Reliable 1 Laboratories LLC,Released
2,May 20 2022 12:39PM,237752,18,HH6056,Hush Hush,Released
3,May 20 2022 12:37PM,237754,18,CLRSPC-20974279,"ClearSpec, LLC",Released
4,May 20 2022 12:35PM,237739,20,ACI-8026484-BO,"ACI Healthcare USA, Inc.",Released
5,May 20 2022 12:35PM,237739,20,ACI-8040384,"ACI Healthcare USA, Inc.",Released
6,May 20 2022 12:35PM,237739,20,8040948,"ACI Healthcare USA, Inc.",Released
7,May 20 2022 12:35PM,237739,20,8040949,"ACI Healthcare USA, Inc.",Released
8,May 20 2022 12:35PM,237739,20,8040950,"ACI Healthcare USA, Inc.",Released
9,May 20 2022 12:35PM,237739,20,8040951,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,237752,Released,1
38,237753,Released,1
39,237754,Released,1
40,237755,Released,1
41,237756,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237752,NaN,NaN,1.0
237753,NaN,NaN,1.0
237754,NaN,NaN,1.0
237755,NaN,NaN,1.0
237756,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3.0,0.0,2.0
237654,0.0,1.0,0.0
237655,0.0,0.0,5.0
237659,0.0,0.0,15.0
237693,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3,0,2
237654,0,1,0
237655,0,0,5
237659,0,0,15
237693,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,0,2
1,237654,0,1,0
2,237655,0,0,5
3,237659,0,0,15
4,237693,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,,2
1,237654,,1,
2,237655,,,5
3,237659,,,15
4,237693,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 20 2022 12:47PM,237756,10,Emerginnova
1,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC
2,May 20 2022 12:39PM,237752,18,Hush Hush
3,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC"
4,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc."
13,May 20 2022 12:34PM,237753,18,"ClearSpec, LLC"
14,May 20 2022 12:32PM,237751,18,"ClearSpec, LLC"
15,May 20 2022 12:28PM,237744,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
26,May 20 2022 12:28PM,237750,18,"ClearSpec, LLC"
27,May 20 2022 12:28PM,237749,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 20 2022 12:47PM,237756,10,Emerginnova,,,1
1,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC,,,1
2,May 20 2022 12:39PM,237752,18,Hush Hush,,,1
3,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC",,,1
4,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc.",,,9
5,May 20 2022 12:34PM,237753,18,"ClearSpec, LLC",,,1
6,May 20 2022 12:32PM,237751,18,"ClearSpec, LLC",,,1
7,May 20 2022 12:28PM,237744,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,6,5
8,May 20 2022 12:28PM,237750,18,"ClearSpec, LLC",,,1
9,May 20 2022 12:28PM,237749,10,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 12:47PM,237756,10,Emerginnova,1,,
1,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC,1,,
2,May 20 2022 12:39PM,237752,18,Hush Hush,1,,
3,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC",1,,
4,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc.",9,,
5,May 20 2022 12:34PM,237753,18,"ClearSpec, LLC",1,,
6,May 20 2022 12:32PM,237751,18,"ClearSpec, LLC",1,,
7,May 20 2022 12:28PM,237744,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5,6,
8,May 20 2022 12:28PM,237750,18,"ClearSpec, LLC",1,,
9,May 20 2022 12:28PM,237749,10,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 12:47PM,237756,10,Emerginnova,1,,
1,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC,1,,
2,May 20 2022 12:39PM,237752,18,Hush Hush,1,,
3,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC",1,,
4,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc.",9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 12:47PM,237756,10,Emerginnova,1.0,NaN,NaN
1,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC,1.0,NaN,NaN
2,May 20 2022 12:39PM,237752,18,Hush Hush,1.0,NaN,NaN
3,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC",1.0,NaN,NaN
4,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc.",9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 12:47PM,237756,10,Emerginnova,1.0,0.0,0.0
1,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC,1.0,0.0,0.0
2,May 20 2022 12:39PM,237752,18,Hush Hush,1.0,0.0,0.0
3,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC",1.0,0.0,0.0
4,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc.",9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3090415,18.0,17.0,0.0
15,713160,2.0,27.0,7.0
16,237742,1.0,0.0,0.0
17,237654,0.0,1.0,0.0
18,1901940,8.0,0.0,0.0
19,475442,10.0,6.0,0.0
20,1663824,34.0,0.0,3.0
21,237724,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3090415,18.0,17.0,0.0
1,15,713160,2.0,27.0,7.0
2,16,237742,1.0,0.0,0.0
3,17,237654,0.0,1.0,0.0
4,18,1901940,8.0,0.0,0.0
5,19,475442,10.0,6.0,0.0
6,20,1663824,34.0,0.0,3.0
7,21,237724,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,18.0,17.0,0.0
1,15,2.0,27.0,7.0
2,16,1.0,0.0,0.0
3,17,0.0,1.0,0.0
4,18,8.0,0.0,0.0
5,19,10.0,6.0,0.0
6,20,34.0,0.0,3.0
7,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,18.0
1,15,Released,2.0
2,16,Released,1.0
3,17,Released,0.0
4,18,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,18,19,20,21
Status,,,,,,,,
Completed,0.0,7.0,0.0,0.0,0.0,0.0,3.0,0.0
Executing,17.0,27.0,0.0,1.0,0.0,6.0,0.0,0.0
Released,18.0,2.0,1.0,0.0,8.0,10.0,34.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,18,19,20,21
0,Completed,0.0,7.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,17.0,27.0,0.0,1.0,0.0,6.0,0.0,0.0
2,Released,18.0,2.0,1.0,0.0,8.0,10.0,34.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,18,19,20,21
0,Completed,0.0,7.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,17.0,27.0,0.0,1.0,0.0,6.0,0.0,0.0
2,Released,18.0,2.0,1.0,0.0,8.0,10.0,34.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()